In [ ]:
# =============================================
# 📦 1. Install & Import Libraries
# =============================================

!pip install -q chromadb sentence-transformers pandas

import os
import pandas as pd
import chromadb
from chromadb.utils import embedding_functions
from IPython.display import display, HTML
import logging

logging.basicConfig(level=logging.INFO)


In [ ]:
# Install necessary packages
!pip install -q chromadb
!pip install -q -U sentence-transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 84.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 80.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 77.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 91.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.8 MB/s eta 0:00:00


In [ ]:
from google.colab import files
import pandas as pd
import io

# Upload CSV file at runtime
uploaded = files.upload()

# Automatically read the uploaded file
for filename in uploaded.keys():
    dataset_path = filename

# Load dataset
df = pd.read_csv(dataset_path)
df.head()


Saving myntra_products_catalog.csv to myntra_products_catalog.csv


,ProductID,ProductName,ProductBrand,Gender,Price (INR),NumImages,Description,PrimaryColor
0,10017413,DKNY Unisex Black & Grey Printed Medium Trolle...,DKNY,Unisex,11745,7,"Black and grey printed medium trolley bag, sec...",Black
1,10016283,EthnoVogue Women Beige & Grey Made to Measure ...,EthnoVogue,Women,5810,7,Beige & Grey made to measure kurta with churid...,Beige
2,10009781,SPYKAR Women Pink Alexa Super Skinny Fit High-...,SPYKAR,Women,899,7,Pink coloured wash 5-pocket high-rise cropped ...,Pink
3,10015921,Raymond Men Blue Self-Design Single-Breasted B...,Raymond,Men,5599,5,Blue self-design bandhgala suitBlue self-desig...,Blue
4,10017833,Parx Men Brown & Off-White Slim Fit Printed Ca...,Parx,Men,759,5,"Brown and off-white printed casual shirt, has ...",White


In [ ]:
df.isna().value_counts()

ProductID  ProductName  ProductBrand  Gender  Price (INR)  NumImages  Description  PrimaryColor
False      False        False         False   False        False      False        False           11597
                                                                                   True              894
Name: count, dtype: int64

In [ ]:
df.fillna("None",inplace=True)

In [ ]:
# Normalize and combine relevant columns
df = df.dropna(subset=['ProductName', 'Description'])

# Create combined document text
df['document'] = (df['ProductName'].astype(str) + ". " + df['Description'].astype(str)).str.lower()

# Create unique IDs
df['uid'] = df.index.astype(str)

# Preview processed data
df[['uid', 'document']].head()




,uid,document
0,0,dkny unisex black & grey printed medium trolle...
1,1,ethnovogue women beige & grey made to measure ...
2,2,spykar women pink alexa super skinny fit high-...
3,3,raymond men blue self-design single-breasted b...
4,4,parx men brown & off-white slim fit printed ca...


In [ ]:
import chromadb
from chromadb.utils import embedding_functions

# Sample 50% of the data to speed up embedding
df_sampled = df.sample(frac=0.3, random_state=42).reset_index(drop=True)

# Create persistent client
chroma_client = chromadb.PersistentClient(path="myantra_vectordb")

# Create embedding function
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="all-mpnet-base-v2"
)

# Create or retrieve collection
collection = chroma_client.get_or_create_collection(
    name="myantra_products",
    embedding_function=sentence_transformer_ef
)

# Add sampled documents to ChromaDB
collection.add(
    documents=df_sampled['document'].tolist(),
    metadatas=df_sampled[['ProductName', 'Description', 'Price (INR)']].to_dict(orient="records"),
    ids=df_sampled['uid'].tolist()
)

print(f"✅ Indexed {len(df_sampled)} documents (30% of original dataset)")



✅ Indexed 3747 documents (30% of original dataset)


In [56]:
def semantic_search(query, n_results=5):
    query = query.lower()  # normalize query
    results = collection.query(
        query_texts=[query],
        n_results=n_results,
        include=["documents", "metadatas", "distances"]
    )

    for i in range(n_results):
        print(f"🔹 Result {i+1}")
        print(f"📦 Title: {results['metadatas'][0][i]['ProductName']}")
        print(f"📝 Description: {results['metadatas'][0][i]['Description']}")
        print(f" price: {results['metadatas'][0][i]['Price (INR)']}")
        print(f"📏 Distance: {results['distances'][0][i]:.4f}")
        print("-" * 60)

# Example usage
semantic_search("collection of ornaments")


🔹 Result 1
📦 Title: AccessHer 22k Gold-Plated Jewellery Set
📝 Description: This jewellery set consists of a necklace and a pair of earrings Gold-plated statement necklace,secured with drawstring fasteningA pair of matching earrings, secured with a post-and-back closure
 price: 645
📏 Distance: 0.8910
------------------------------------------------------------
🔹 Result 2
📦 Title: AccessHer Gold-Plated & Red Enamelled Geometric Drop Earrings
📝 Description: Gold-Plated and red enamelled geometric drop earrings Secured with a post and back
 price: 577
📏 Distance: 0.9148
------------------------------------------------------------
🔹 Result 3
📦 Title: OBSESSIONS Set of 4 White & Red Solid Ceramic Bath Accessories
📝 Description: Set content: 1 soap dispenser, 1 soap dish, 1 tumbler and 1 brush holderColour: White & redMaterial: CeramicPattern: Solid
 price: 740
📏 Distance: 0.9221
------------------------------------------------------------
🔹 Result 4
📦 Title: Priyaasi Women Set Of 8 Red Gold-

In [57]:
semantic_search("brown shoe having lace")

🔹 Result 1
📦 Title: ID Men Brown Solid Leather Mid-Top Sneakers
📝 Description: A pair of round-toe brown sneakers, has mid-top styling, lace-up detailLeather upperCushioned footbedTextured and patterned outsoleWarranty: 3 monthsWarranty provided by brand/manufacturer
 price: 1286
📏 Distance: 0.8019
------------------------------------------------------------
🔹 Result 2
📦 Title: ID Men Brown Shoe-Style Leather Sandals
📝 Description: A pair of brown shoe-style sandalsLeather upper with velcro closureCushioned footbedPatterned synthetic outsoleWarranty: 3 monthsWarranty provided by brand/manufacturer
 price: 2385
📏 Distance: 0.8516
------------------------------------------------------------
🔹 Result 3
📦 Title: Levis Men Brown Sneakers
📝 Description: A pair of round-toe brown sneakers, has regular styling, lace-up detailSynthetic Leather upperCushioned footbedTextured and patterned outsoleWarranty: 3 monthsWarranty provided by brand/manufacturer
 price: 3999
📏 Distance: 0.8564
-----------

In [ ]:
semantic_search("light jacket for trekking in cold")

🔹 Result 1
📦 Title: Indian Terrain Men Black Solid Padded Jacket
📝 Description: Black solid padded jacket, has a mock collar, 5 pockets, zip closure, long sleeves, straight hem, and polyester
 price: 2699
📏 Distance: 1.0378
------------------------------------------------------------
🔹 Result 2
📦 Title: Indian Terrain Men Brown Solid Lightweight Biker Jacket
📝 Description: Brown solid jacket, has a stand collar, 2 pockets, zip closure, long sleeves, straight hem, and polyester lining with a pocket
 price: 2699
📏 Distance: 1.0540
------------------------------------------------------------
🔹 Result 3
📦 Title: WROGN Men Black Solid Biker Jacket
📝 Description: Black solid jacket, has a stand collar, 2 pockets, zip closure, long sleeves, straight hem
 price: 2249
📏 Distance: 1.0867
------------------------------------------------------------
🔹 Result 4
📦 Title: Indian Terrain Men Black Rust Red Solid Reversible Insulator Puffer Jacket
📝 Description: Black and Rust Red solid reversible jack

In [ ]:
semantic_search("premium-looking outfit under budget for men")

🔹 Result 1
📦 Title: Next Look Men Red Slim Fit Solid Formal Shirt
📝 Description: Red solid casual shirt, has a spread collar, long sleeves, button placket, straight hem, and 1 pocket
 price: 549
📏 Distance: 0.8989
------------------------------------------------------------
🔹 Result 2
📦 Title: Next Look Men Blue Slim Fit Solid Formal Shirt
📝 Description: Blue solid formal shirt, has a spread collar, long sleeves, button placket, straight hem, and 1 patch pocket
 price: 499
📏 Distance: 0.9161
------------------------------------------------------------
🔹 Result 3
📦 Title: Next Look Men Blue & Black Slim Fit Printed Formal Shirt
📝 Description: Blue and Black printed formal shirt, has a spread collar, long sleeves, button placket, straight hem, and 1 patch pocket
 price: 649
📏 Distance: 0.9252
------------------------------------------------------------
🔹 Result 4
📦 Title: Next Look Men Pink Slim Fit Solid Formal Shirt
📝 Description: Pink solid formal shirt, has a spread collar, long slee

In [ ]:
semantic_search("shoes for gym and running for men")

🔹 Result 1
📦 Title: Campus Men Grey Mesh Running Shoes
📝 Description: A pair of grey running sports shoes, has regular styling, lace-up detailMesh upperCushioned footbedTextured and patterned outsoleForest-2 with it's minimal design is made of lightweight, single layer mesh for breathability and soft foam beneath the foot for revolutionary comfort and responsiveness. Dynamic fit upper and full length rubber out-sole deliver responsive cushioning and durable traction. It's molded pods offer multisurface traction. Pods flatten on impact then spring back at takeoff creating a piston effect that delivers responsive cushioning.Warranty: 30 daysWarranty provided by brand/manufacturer
 price: 799
📏 Distance: 0.7736
------------------------------------------------------------
🔹 Result 2
📦 Title: Reebok Men Black & Grey Woven Design Speed Flexweave Training Soes
📝 Description: Special technologiesLow-cut design for freedom of motion and quick transitionsDuracage heel clip for added stabilitySof